In [8]:
import json
import pandas as pd
from collections import Counter

In [9]:
path = "/Users/admin/PycharmProjects/llm_rag_bot/data/python_faq_dataset_enhanced.jsonl"

rows = []
with open(path, "r", encoding="utf-8") as f:
    for line in f:
        rows.append(json.loads(line))

df = pd.DataFrame(rows)
df

,question,answer,source,subsource,id,metadata
0,What is Python?,"Python is an interpreted, interactive, object-...",General Python FAQ,General Information,general information_0001,"{'tags': ['what', 'general information', 'pyth..."
1,What is the Python Software Foundation?,The Python Software Foundation is an independe...,General Python FAQ,General Information,general information_0002,"{'tags': ['software', 'foundation', 'general i..."
2,Are there copyright restrictions on the use of...,"You can do anything you want with the source, ...",General Python FAQ,General Information,general information_0003,"{'tags': ['there', 'copyright', 'general infor..."
3,What were the key inspirations and experiences...,Guido van Rossum drew inspiration from his exp...,General Python FAQ,General Information,general information_0004,"{'tags': ['experiences', 'inspirations', 'that..."
4,What practical problem directly prompted the c...,The immediate catalyst was Guido's work on the...,General Python FAQ,General Information,general information_0005,"{'tags': ['prompted', 'practical', 'creation',..."
...,...,...,...,...,...,...
363,What alternative ordering does Jim Roskind sug...,Jim Roskind suggests structuring each module i...,Programming FAQ,Modules,modules_0007,"{'tags': ['handle', 'ordering', 'does', 'sugge..."
364,How can I dynamically import a submodule (like...,"Use `importlib.import_module('x.y.z')`, which ...",Programming FAQ,Modules,modules_0008,"{'tags': ['import', 'deepest', 'module', 'refe..."
365,Why do changes to an imported module not take ...,Python caches imported modules in `sys.modules...,Programming FAQ,Modules,modules_0009,"{'tags': ['after', 'module', 'imported', 'effe..."
366,What are the limitations of using importlib.re...,`reload()` has several limitations: 1) It does...,Programming FAQ,Modules,modules_0010,"{'tags': ['what', 'using', 'limitations', 'mod..."


In [4]:
stats = {
    "num_rows": len(df),
    "columns": df.columns.tolist(),
    "id_unique": df["id"].nunique() if "id" in df else None,
    "empty_questions": int(df["question"].eq("").sum()) if "question" in df else None,
    "empty_answers": int(df["answer"].eq("").sum()) if "answer" in df else None,
}

df["q_len"] = df["question"].str.len()
df["a_len"] = df["answer"].str.len()

In [5]:
df

,question,answer,source,subsource,id,metadata,q_len,a_len
0,What is Python?,"Python is an interpreted, interactive, object-...",General Python FAQ,General Information,general information_0001,"{'tags': ['what', 'general information', 'pyth...",15,943
1,What is the Python Software Foundation?,The Python Software Foundation is an independe...,General Python FAQ,General Information,general information_0002,"{'tags': ['software', 'foundation', 'general i...",39,490
2,Are there copyright restrictions on the use of...,"You can do anything you want with the source, ...",General Python FAQ,General Information,general information_0003,"{'tags': ['there', 'copyright', 'general infor...",54,762
3,What were the key inspirations and experiences...,Guido van Rossum drew inspiration from his exp...,General Python FAQ,General Information,general information_0004,"{'tags': ['experiences', 'inspirations', 'that...",79,303
4,What practical problem directly prompted the c...,The immediate catalyst was Guido's work on the...,General Python FAQ,General Information,general information_0005,"{'tags': ['prompted', 'practical', 'creation',...",64,362
...,...,...,...,...,...,...,...,...
363,What alternative ordering does Jim Roskind sug...,Jim Roskind suggests structuring each module i...,Programming FAQ,Modules,modules_0007,"{'tags': ['handle', 'ordering', 'does', 'sugge...",78,341
364,How can I dynamically import a submodule (like...,"Use `importlib.import_module('x.y.z')`, which ...",Programming FAQ,Modules,modules_0008,"{'tags': ['import', 'deepest', 'module', 'refe...",100,215
365,Why do changes to an imported module not take ...,Python caches imported modules in `sys.modules...,Programming FAQ,Modules,modules_0009,"{'tags': ['after', 'module', 'imported', 'effe...",82,312
366,What are the limitations of using importlib.re...,`reload()` has several limitations: 1) It does...,Programming FAQ,Modules,modules_0010,"{'tags': ['what', 'using', 'limitations', 'mod...",53,379


In [7]:
length_stats = {
    "q_len_min": df["q_len"].min(),
    "q_len_max": df["q_len"].max(),
    "q_len_mean": df["q_len"].mean(),
    "a_len_min": df["a_len"].min(),
    "a_len_max": df["a_len"].max(),
    "a_len_mean": df["a_len"].mean(),
}

(stats, length_stats)

({'num_rows': 368,
  'columns': ['question', 'answer', 'source', 'subsource', 'id', 'metadata'],
  'id_unique': 368,
  'empty_questions': 0,
  'empty_answers': 0},
 {'q_len_min': np.int64(15),
  'q_len_max': np.int64(155),
  'q_len_mean': np.float64(70.19565217391305),
  'a_len_min': np.int64(20),
  'a_len_max': np.int64(998),
  'a_len_mean': np.float64(368.9945652173913)})

Датасет консистентный, все строки парсятся, id все униклаьные. Есть ответы длиной до 1000 символов. Пока оставим так, надеюсь RAG пройдет без проблем. 
Но скорее всего придется улучшать датасет, разделяя чанки. Также можно будет улучшить metadata путем извлечения ключевых слов, категорий и тд.

### Подготовка для RAG-пайплайна